# INVALID_TOOL_RESULTS

You are passing too many, too few, or mismatched [`ToolMessages`](https://api.js.langchain.com/classes/_langchain_core.messages_tool.ToolMessage.html) to a model.

When [using a model to call tools](/docs/concepts#functiontool-calling), the [`AIMessage`](https://api.js.langchain.com/classes/_langchain_core.messages.AIMessage.html)
the model responds with will contain a `tool_calls` array. To continue the flow, the next messages you pass back to the model must
be exactly one `ToolMessage` for each item in that array containing the result of that tool call. Each `ToolMessage` must have a `tool_call_id` field
that matches one of the `tool_calls` on the `AIMessage`.

For example, given the following response from a model:

In [14]:
from typing import List
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage


model = ChatOpenAI(model="gpt-4o-mini")


@tool
def foo_tool() -> str:
    """
    A dummy tool that returns 'action complete!'
    """
    return "action complete!"

model_with_tools = model.bind_tools([foo_tool])

chat_history: List[BaseMessage] = [
    HumanMessage(content='Call tool "foo" twice with no arguments')
]

response_message = model_with_tools.invoke(chat_history)

print(response_message.tool_calls)

[{'name': 'foo_tool', 'args': {}, 'id': 'call_GeBhcpR3Cc4sDO6PMjXp80wy', 'type': 'tool_call'}, {'name': 'foo_tool', 'args': {}, 'id': 'call_zyXhlqdhKJL0OTaq82bcUvdz', 'type': 'tool_call'}]


Calling the model with only one tool response would result in an error:

In [16]:
from langchain_core.messages import AIMessage, ToolMessage

tool_call = response_message.tool_calls[0]
tool_response = foo_tool.invoke(tool_call)

chat_history.append(AIMessage(content=response_message.content, additional_kwargs=response_message.additional_kwargs))
chat_history.append(ToolMessage(content=str(tool_response), tool_call_id=tool_call.get('id')))

final_response = model_with_tools.invoke(chat_history)
print(final_response)

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_zyXhlqdhKJL0OTaq82bcUvdz", 'type': 'invalid_request_error', 'param': 'messages.[3].role', 'code': None}}

If we add a second response, the call will succeed as expected because we now have one tool response per tool call:

In [3]:
const toolResponse2 = await dummyTool.invoke(responseMessage.tool_calls![1])

chatHistory.push(toolResponse2)

await modelWithTools.invoke(chatHistory)

AIMessage {
  "id": "chatcmpl-AIgTPDBm1epnnLHx0tPFTgpsf8Ay6",
  "content": "The tool \"foo\" was called twice, and both times returned the result: \"action complete!\".",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 98,
      "completionTokens": 21,
      "totalTokens": 119
    },
    "finish_reason": "stop",
    "usage": {
      "prompt_tokens": 98,
      "completion_tokens": 21,
      "total_tokens": 119,
      "prompt_tokens_details": {
        "cached_tokens": 0
      },
      "completion_tokens_details": {
        "reasoning_tokens": 0
      }
    },
    "system_fingerprint": "fp_e2bde53e6e"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 21,
    "input_tokens": 98,
    "total_tokens": 119,
    "input_token_details": {
      "cache_read": 0
    },
    "output_token_details": {
      "reasoning": 0
    }
  }
}


But if we add a duplicate, extra tool response, the call will fail again:

In [4]:
const duplicateToolResponse2 = await dummyTool.invoke(responseMessage.tool_calls![1])

chatHistory.push(duplicateToolResponse2)

await modelWithTools.invoke(chatHistory)

BadRequestError: 400 Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'.
    at APIError.generate (/Users/jacoblee/langchain/langchainjs/libs/langchain-openai/node_modules/openai/error.js:45:20)
    at OpenAI.makeStatusError (/Users/jacoblee/langchain/langchainjs/libs/langchain-openai/node_modules/openai/core.js:291:33)
    at OpenAI.makeRequest (/Users/jacoblee/langchain/langchainjs/libs/langchain-openai/node_modules/openai/core.js:335:30)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async /Users/jacoblee/langchain/langchainjs/libs/langchain-openai/dist/chat_models.cjs:1441:29
    at async RetryOperation._fn (/Users/jacoblee/langchain/langchainjs/node_modules/p-retry/index.js:50:12) {
  status: 400,
  headers: {
    'access-control-expose-headers': 'X-Request-ID',
    'alt-svc': 'h3=":443"; ma=86400',
    'cf-cache-status': 'DYNAMIC',
    'cf-ray': '8d31d57dff5e0f3b-EWR',
    connection:

You should additionally not pass `ToolMessages` back to to a model if they are not preceded by an `AIMessage` with tool calls. For example, this will fail:

In [5]:
await modelWithTools.invoke([{
  role: "tool",
  content: "action completed!",
  tool_call_id: "dummy",
}])

BadRequestError: 400 Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'.
    at APIError.generate (/Users/jacoblee/langchain/langchainjs/libs/langchain-openai/node_modules/openai/error.js:45:20)
    at OpenAI.makeStatusError (/Users/jacoblee/langchain/langchainjs/libs/langchain-openai/node_modules/openai/core.js:291:33)
    at OpenAI.makeRequest (/Users/jacoblee/langchain/langchainjs/libs/langchain-openai/node_modules/openai/core.js:335:30)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async /Users/jacoblee/langchain/langchainjs/libs/langchain-openai/dist/chat_models.cjs:1441:29
    at async RetryOperation._fn (/Users/jacoblee/langchain/langchainjs/node_modules/p-retry/index.js:50:12) {
  status: 400,
  headers: {
    'access-control-expose-headers': 'X-Request-ID',
    'alt-svc': 'h3=":443"; ma=86400',
    'cf-cache-status': 'DYNAMIC',
    'cf-ray': '8d31d5da7fba19aa-EWR',
    connection:

See [this guide](/docs/how_to/tool_results_pass_to_model/) for more details on tool calling.

## Troubleshooting

The following may help resolve this error:

- If you are using a custom executor rather than a prebuilt one like LangGraph's [`ToolNode`](https://langchain-ai.github.io/langgraphjs/reference/classes/langgraph_prebuilt.ToolNode.html)
  or the legacy LangChain [AgentExecutor](/docs/how_to/agent_executor), verify that you are invoking and returning the result for one tool per tool call.
- If you are using [few-shot tool call examples](/docs/how_to/tools_few_shot) with messages that you manually create, and you want to simulate a failure,
  you still need to pass back a `ToolMessage` whose content indicates that failure.
